## 🧩 Overview — Agentic Commerce Multi-Agent System

This notebook demonstrates an **agentic commerce pipeline** — a simulated multi-agent system that automates product discovery, pricing, content generation, supervision, and transaction validation with trust at its core.  

It showcases how **AI agents collaborate** to plan, optimize, and execute a commerce flow end-to-end while using **guardrails** to ensure safety, transparency, and trust.  

When complete, the system compiles a detailed JSON report capturing the essential data of this new *agentic commerce flow* that can then be passes on the payment transaction system.
- 🧩 **Agent Names** – which AI module handled each decision stage  
- 💭 **Intent** – the user’s shopping request in structured form  
- 💰 **Budget** – the detected spending limit and the system’s cost estimates  
- 🛡️ **Guardrail Status** – pass or fail, depending on ethical and budget adherence  


The agents involved include:
- **Intent Planner Agent** – understands user goals and budgets  
- **Product Discovery Agent** – finds relevant products using Tavily search  
- **Price Optimizer Agent** – estimates costs and ensures they fit within budget  
- **Commerce Copywriter Agent** – creates persuasive, ethical ad copy  
- **Supervisor Agent** – validates quality, compliance, and safety  
- **Transaction Agent** – simulates payment handling  
- **Checkout Simulator** – mimics a checkout and shows if the process succeeds  
- **Trust Guardrails Agent** – checks ethical and budgetary consistency  

👉 The saved example run shows a case where **the guardrail fails**, meaning the transaction is **blocked before payment** — demonstrating how the system prevents untrustworthy or unsafe operations.  
Some modules (like web search and transactions) are **simulated** for demonstration purposes.

---

## 1. Environment Setup
This cell installs dependencies, loads API keys (OpenAI, Tavily), and initializes the session for the multi-agent orchestration.


In [53]:
# 💳 Agentic Commerce Demo —
# End-to-end multi-agent orchestration with trust, guardrails, and simulated transactions.

# =====================================================
# 1. SETUP
# =====================================================
!pip install aisuite[openai] -q
!pip install tavily-python -q
!pip install aisuite tavily-python python-dotenv -q


import os, re, json, requests
from datetime import datetime
from tavily import TavilyClient
import aisuite
from dotenv import load_dotenv
from IPython.display import display, HTML
from google.colab import userdata
import random


# Session
session = requests.Session()
session.headers.update({"User-Agent": "LF-Agent/1.0 (mailto:you@example.com)"})

# Keys
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")
load_dotenv()
client = aisuite.Client()
print("✅ Environment ready.")

✅ Environment ready.


## 2. Utilities & Guardrails

This section defines helper functions for:
- Displaying agent outputs and tool results in styled HTML panels
- Logging tool calls and summaries

It also introduces the **Trust Guardrails Agent**, which enforces safety and trust principles in commerce:
- Checks that transactions stay within the user’s budget  
- Ensures the supervisor has approved the pipeline  
- Validates that copywriter content includes a clear, ethical call-to-action  
- Flags unsafe, biased, or non-compliant output  

When any guardrail fails, the transaction flow is blocked — this prevents unsafe or financially inconsistent outcomes.


In [54]:



# =====================================================
# 2. UTILITIES & GUARDRAILS
# =====================================================
def log_agent_title_html(title, icon="🤖"):
    display(HTML(f"""
    <div style="padding:1em;margin:1em 0;background-color:#f0f4f8;border-left:6px solid #1976D2;">
      <h2 style="margin:0;color:#0D47A1;">{icon} {title}</h2>
    </div>"""))

def log_final_summary_html(content):
    display(HTML(f"""
      <div style="border-left:4px solid #2E7D32;padding:1em;margin:1em 0;
                  background-color:#e8f5e9;color:#1B5E20;">
        <strong>✅ Output:</strong>
        <pre style="white-space:pre-wrap;">{content}</pre>
      </div>"""))

def log_tool_call_html(tool_name, args):
    display(HTML(f"""
    <div style="background:#e3f2fd;padding:10px;border-left:5px solid #1976D2;margin:8px 0;">
    📞 <b>Tool Call:</b> {tool_name} → {args}
    </div>"""))

def log_tool_result_html(result):
    display(HTML(f"""
    <div style="background:#ffffff;
                padding:10px;
                border-left:5px solid #4CAF50;
                margin:8px 0;">
      <b style="color:#2E7D32;">✅ Tool Result:</b><br>
      <pre style="color:#212121;white-space:pre-wrap;
                  font-size:14px;">{json.dumps(result, indent=2)}</pre>
    </div>"""))



def trust_guardrails(copy_output, user_budget, price_optimizer_output=None, supervisor_output=None):
    """
    🛡️ Trust Guardrails Agent
    Ensures the pipeline maintains user trust and financial consistency.
    Checks:
      1. Budget adherence (from Price Optimizer)
      2. Supervisor approval
      3. Copywriter truthfulness / clear CTA
    """

    log_agent_title_html("🛡️ Trust Guardrails Agent", "🧭")

    issues = []
    passed = True

    # 1️⃣ Budget Guardrail — must match budget limit
    total_price = 0
    if price_optimizer_output:
        total_price = price_optimizer_output.get("estimated_total", 0)
        if total_price > user_budget:
            passed = False
            issues.append(
                f"Estimated total ${total_price} exceeds user budget ${user_budget}."
            )

    # 2️⃣ Supervisor approval check
    if supervisor_output and supervisor_output.get("status", "").lower() not in ["approved", "ok", "pass"]:
        passed = False
        issues.append("Supervisor did not approve the pipeline execution.")

    # 3️⃣ Copywriter call-to-action (CTA) validation
    quote_text = ""
    if isinstance(copy_output, dict):
        quote_text = copy_output.get("quote", "")
    elif isinstance(copy_output, str):
        quote_text = copy_output

    if quote_text and "buy" not in quote_text.lower():
        passed = False
        issues.append("Copywriter text does not include a clear call-to-action to purchase.")

    # 4️⃣ Default pass message
    if not issues:
        issues.append("All checks passed. No issues detected.")

    # 5️⃣ Styling
    color = "#2E7D32" if passed else "#C62828"  # green or red
    bg = "#E8F5E9" if passed else "#FFEBEE"     # light green or red
    emoji = "✅" if passed else "⚠️"

    # 6️⃣ Bright, readable display panel
    display(HTML(f"""
    <div style="border-left:6px solid {color};
                padding:16px;
                margin:12px 0;
                background-color:{bg};
                color:#111;
                font-family:'Segoe UI',sans-serif;
                line-height:1.5;
                box-shadow:0 2px 6px rgba(0,0,0,0.08);
                border-radius:8px;">
      <h3 style="margin-top:0;
                 color:{color};
                 font-size:18px;">
        {emoji} Trust Guardrails Summary
      </h3>
      <ul style="margin-top:8px;
                 color:#000;
                 font-size:14px;">
        {''.join([f'<li>{i}</li>' for i in issues])}
      </ul>
      <p style="margin-top:10px;
                font-weight:600;
                font-size:15px;
                color:{color};">
        Status: {"PASS" if passed else "FAIL"}
      </p>
      <p style="margin-top:4px;font-size:13px;color:#555;">
        <b>Verified Total:</b> ${total_price} | <b>User Budget:</b> ${user_budget}
      </p>
    </div>
    """))

    return passed, issues





## 3. Tavily Search Tool

This defines a simple search wrapper around the **Tavily API**, which retrieves structured web results.  
It’s used by the Product Discovery Agent to find shoppable products from sources like Amazon, IKEA, Wayfair, and Target.  
All API responses are cleaned and returned as lists of title–content–URL objects, optionally with images.


In [55]:

# =====================================================
# 3. TAVILY SEARCH TOOL
# =====================================================
def tavily_search_tool(query: str, max_results: int = 5, include_images: bool = False):
    api_key = os.getenv("TAVILY_API_KEY")
    client = TavilyClient(api_key=api_key)
    try:
        resp = client.search(query=query, max_results=max_results, include_images=include_images)
        results = []
        for r in resp.get("results", []):
            results.append({
                "title": r.get("title", ""),
                "content": r.get("content", ""),
                "url": r.get("url", "")
            })
        if include_images:
            for img_url in resp.get("images", []):
                results.append({"image_url": img_url})
        return results
    except Exception as e:
        return [{"error": str(e)}]


## 4. Agent Definitions

This section defines all **core agents** in the pipeline:

- **Intent Planner Agent** – Extracts structured shopping intent and budget from a user’s request.  
- **Product Discovery Agent** – Uses Tavily search to find specific items matching style, category, and budget.  
- **Price Optimizer Agent** – Simulates price estimation and checks whether total cost fits within budget.  
- **Commerce Copywriter Agent** – Generates short, persuasive purchase copy emphasizing trust and verified sellers.  
- **Supervisor Agent** – Reviews outputs for safety, completeness, and compliance.  
- **Transaction Agent** – Simulates a payment approval step (only if all guardrails pass).  
- **Checkout Simulator** – Displays a simulated receipt; if a guardrail fails, the transaction is marked as **blocked**.  

Each agent is displayed through rich HTML summaries to visualize its decision and reasoning process.


In [56]:


# =====================================================
# 4. AGENTS
# =====================================================

# --- Intent Planner ---
def extract_budget_from_text(user_text):
    match = re.search(r"\$?\s*([0-9]+(?:\.[0-9]+)?)(?:\s*k|K)?", user_text)
    if match:
        val = match.group(1)
        return int(float(val) * 1000) if 'k' in user_text.lower() else int(float(val))
    return 500

def intent_planner_agent(user_request):
    log_agent_title_html("Intent Planner Agent", "🧭")
    budget = extract_budget_from_text(user_request)
    prompt = f"""
You are an e-commerce intent planner.
Extract structured intent in JSON format.

User request: "{user_request}"

Return JSON:
{{
  "goal": "...",
  "category": "...",
  "budget": {budget},
  "style": ["..."]
}}
"""
    response = client.chat.completions.create(
        model="openai:o4-mini",
        messages=[{"role": "user", "content": prompt}]
    )
    msg = response.choices[0].message.content.strip()
    log_final_summary_html(msg)
    try:
        match = re.search(r"\{.*\}", msg, re.DOTALL)
        parsed = json.loads(match.group(0))
    except:
        parsed = {"goal": "shopping", "category": "home", "budget": budget, "style": ["modern"]}
    return parsed





def price_optimizer_agent(products, intent_json):
    log_agent_title_html("💰 Price Optimizer Agent", "📊")

    # Extract budget
    budget = intent_json.get("budget", 0)

    # Example: infer or simulate prices
    prices = []
    total_price = 0
    for p in products:
        price = random.randint(90, 150)  # Replace with real parsed price later
        prices.append(price)
        total_price += price

    within_budget = total_price <= budget

    summary = {
        "agent_name": "Price Optimizer Agent",
        "estimated_total": total_price,
        "individual_prices": prices,
        "within_budget": within_budget,
        "item_count": len(products),
        "budget": budget,
    }

    display(HTML(f"""
    <div style="background-color:#E3F2FD;padding:12px;
                border-left:5px solid #2196F3;
                margin:8px 0;
                color:#000;font-family:'Segoe UI',sans-serif;">
      <h3 style="margin-top:0;">💰 Price Optimizer Summary</h3>
      <p><b>Estimated total:</b> ${total_price}</p>
      <p><b>Budget:</b> ${budget} — {"✅ Within" if within_budget else "⚠️ Over"} budget</p>
      <p style="font-size:14px;color:#444;">Prices derived consistently for downstream agents.</p>
    </div>
    """))

    return summary



def product_discovery_agent(intent_json):
    """
    Product Discovery Agent
    - Infers style, category, and budget from the user's intent
    - Queries Tavily for purchasable furniture products
    - Filters out non-product or category-level pages
    - Displays a readable HTML summary with count and context
    """
    log_agent_title_html("🛒 Product Discovery Agent", "🪑")

    # 1️⃣ Extract budget and intent safely
    budget = intent_json.get("budget", 0)
    user_intent_text = intent_json.get("intent", "").lower()

    # 2️⃣ Infer product category from intent
    if "office" in user_intent_text:
        category = "home office furniture"
    elif "dining" in user_intent_text:
        category = "dining room furniture"
    elif "bedroom" in user_intent_text:
        category = "bedroom furniture"
    elif "living" in user_intent_text:
        category = "living room furniture"
    else:
        category = "furniture"

    # 3️⃣ Infer design style
    if "scandinavian" in user_intent_text:
        style = "scandinavian"
    elif "modern" in user_intent_text:
        style = "modern"
    elif "minimal" in user_intent_text or "minimalist" in user_intent_text:
        style = "minimalist"
    elif "boho" in user_intent_text:
        style = "boho"
    else:
        style = "contemporary"

    # 4️⃣ Build focused Tavily query — target product-level results
    query = (
        f"buy {style} {category} product details price "
        "site:amazon.com OR site:ikea.com OR site:wayfair.com OR site:target.com "
        "product OR item OR sku OR add to cart"
    )

    # 5️⃣ Perform Tavily search
    log_tool_call_html("tavily_search_tool", query)
    results = tavily_search_tool(query, max_results=10, include_images=True)

    # 6️⃣ Filter for true product pages (not category/search listings)
    shopping_domains = ["amazon.com", "ikea.com", "wayfair.com", "target.com"]
    filtered = []
    for r in results:
        url = r.get("url", "")
        title = r.get("title", "").lower()

        # Exclude listings, collections, guides, or short URLs
        if (
            any(domain in url for domain in shopping_domains)
            and not any(
                bad in url
                for bad in [
                    "search",
                    "collections",
                    "category",
                    "departments",
                    "list",
                    "browse",
                    "rooms",
                    "inspiration",
                    "ideas",
                    "tips",
                    "guide",
                    "video",
                ]
            )
            and len(url.split("/")) > 4  # deeper URLs indicate product pages
        ):
            filtered.append(r)

    # 7️⃣ HTML summary display
    display(HTML(f"""
    <div style="background-color:#FFF3E0;
                padding:12px;
                border-left:5px solid #000;
                margin:8px 0;
                color:#000;
                font-family:'Segoe UI',sans-serif;">
        <h3 style="margin-top:0;">🛍️ Product Discovery Summary</h3>
        <p>Found <b>{len(filtered)}</b> individual <i>{style} {category}</i> items within your budget of ${budget}.</p>
        <p style="font-size:14px;color:#444;">(Showing product-level links only — category pages filtered out.)</p>
    </div>
    """))

    # 8️⃣ Return clean structured list
    return filtered




def commerce_copywriter_agent(intent_json, products, budget_info):
    log_agent_title_html("Copywriter Agent", "✍️")

    prompt = f"""
You are an AI commerce copywriter working within a trusted agentic checkout system.
Your job is to create an engaging, inclusive, and concise campaign paragraph that:

1. Summarizes the offer clearly.
2. Encourages the user to complete their purchase right now through the agent (not external links).
3. Builds trust by emphasizing verified sellers, secure payments, and instant checkout.
4. Keeps under 100 words.
5. Matches the tone: warm, professional, empowering.

User intent:
Goal: {intent_json['goal']}
Category: {intent_json['category']}
Budget: ${intent_json['budget']}
Products: {[p['title'] for p in products[:3]]}

Return only the text paragraph — no JSON.
"""

    response = client.chat.completions.create(
        model="openai:o4-mini",
        messages=[{"role": "user", "content": prompt}]
    )

    text = response.choices[0].message.content.strip()
    log_final_summary_html(text)
    return text



# --- Supervisor Agent ---
def supervisor_agent(intent_json, products, budget_info, copy_text):
    log_agent_title_html("Supervisor Agent", "🧑‍💼")
    system_msg = "You are a QA reviewer ensuring outputs are safe, relevant, and complete."
    user_msg = f"""
Review:
Intent: {json.dumps(intent_json, indent=2)}
Budget: {json.dumps(budget_info, indent=2)}
Copy: {copy_text}

Return JSON:
{{
  "status": "pass/fail",
  "issues": ["list of issues"],
  "executive_summary": "2–3 sentence overview"
}}
"""
    res = client.chat.completions.create(
        model="openai:o4-mini",
        messages=[{"role": "system", "content": system_msg},
                  {"role": "user", "content": user_msg}]
    )
    msg = res.choices[0].message.content.strip()
    log_final_summary_html(msg)
    try:
        match = re.search(r"\{.*\}", msg, re.DOTALL)
        parsed = json.loads(match.group(0))
    except:
        parsed = {"status": "unknown", "issues": [], "executive_summary": "Could not parse review."}
    parsed["agent_name"] = "Supervisor Agent"
    return parsed


def transaction_agent(intent, budget_data, products, copy):
    log_agent_title_html("💳 Transaction Agent", "💼")

    total = budget_data["estimated_total"]  # ✅ use single source of truth
    within_budget = budget_data["within_budget"]

    transaction_summary = {
        "agent_name": "Transaction Agent",
        "status": "pending" if not within_budget else "approved",
        "total_amount": total,
        "item_count": budget_data["item_count"],
        "within_budget": within_budget,
    }

    display(HTML(f"""
    <div style="background-color:#FFF8E1;padding:12px;
                border-left:5px solid #000;
                margin:8px 0;
                color:#000;font-family:'Segoe UI',sans-serif;">
      <h3 style="margin-top:0;">💳 Transaction Summary</h3>
      <p><b>Total Amount:</b> ${total}</p>
      <p><b>Items:</b> {budget_data["item_count"]}</p>
      <p><b>Budget Status:</b> {"✅ Within" if within_budget else "⚠️ Over"} budget</p>
    </div>
    """))

    return transaction_summary





def checkout_simulator(transaction, supervisor):
    log_agent_title_html("🧾 Checkout Simulator", "🛍️")

    total = transaction.get("total_amount", 0)  # ✅ pull from transaction agent
    within_budget = transaction.get("within_budget", True)

    color = "#2E7D32" if within_budget else "#C62828"
    bg = "#E8F5E9" if within_budget else "#FFEBEE"

    display(HTML(f"""
    <div style="border-left:5px solid {color};padding:16px;margin:10px 0;
                background-color:{bg};color:#000;font-family:'Segoe UI',sans-serif;">
      <h3 style="margin-top:0;">🧾 Agentic Checkout Receipt</h3>
      <p><b>Total:</b> ${total}</p>
      <p><b>Supervisor:</b> {supervisor.get("status", "N/A")}</p>
      <p><b>Status:</b> {"✅ Completed" if within_budget else "❌ Blocked (Over Budget)"}</p>
    </div>
    """))

    return {
        "agent_name": "Checkout Simulator",
        "status": "completed" if within_budget else "blocked",
        "total": total,
    }



## 5. Packaging & Reporting

This cell defines the **Packaging Agent**, which creates a structured HTML report summarizing:
- User goal and budget  
- Discovered products  
- Generated marketing copy  
- Supervisor’s feedback and overall pipeline result  

It visually ties together all agent outputs to form an **end-to-end commerce experience report**.


In [57]:

# =====================================================
# 5. PACKAGING + ORCHESTRATION
# =====================================================
def packaging_agent(intent_json, products, budget_info, copy_text, supervisor_report):
    log_agent_title_html("Packaging Agent", "📦")
    cards = ""
    for p in products:
        img_html = f'<img src="{p["image"]}" style="width:160px;border-radius:8px;">' if p.get("image") else ""
        cards += f"""
        <div style="display:flex;align-items:center;gap:12px;border:1px solid #ddd;
                    padding:8px;margin-bottom:10px;border-radius:8px;">
            {img_html}
            <div><b>{p['title']}</b><br><small>{p['content']}</small><br>
            <a href="{p['url']}" target="_blank">View</a></div>
        </div>"""
    html = f"""
    <div style="font-family:Arial;max-width:800px;margin:auto;">
      <h2>🛍️ Agentic Commerce Report</h2>
      <p><b>Goal:</b> {intent_json['goal']} — <b>Budget:</b> ${intent_json['budget']}</p>
      {cards}
      <h3>Copywriter Summary</h3><p>{copy_text}</p>
      <h3>Supervisor QA</h3>
      <p><b>Status:</b> {supervisor_report['status']}</p>
      <blockquote>{supervisor_report['executive_summary']}</blockquote>
      <hr><small>Generated {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}</small>
    </div>"""
    display(HTML(html))
    return html



## 6. Full Orchestration Pipeline

This is the main orchestrator function — `run_agentic_commerce_pipeline(user_request)` — which ties all agents together in sequence:

1. Intent understanding  
2. Product discovery  
3. Price estimation  
4. Copy generation  
5. Supervision and safety review  
6. Trust guardrail validation  
7. Transaction and checkout simulation  
8. Packaging and report creation  

At the end, it produces a JSON report and displays a summary of the pipeline’s execution.  
If the guardrails fail, the pipeline halts the transaction before payment is simulated.


In [61]:
from datetime import datetime
import json
from IPython.display import display, HTML

def run_agentic_commerce_pipeline(user_request):
    # 1️⃣ Step-by-step orchestration
    intent = intent_planner_agent(user_request)
    products = product_discovery_agent(intent)
    budget = price_optimizer_agent(products, intent)
    copy = commerce_copywriter_agent(intent, products, budget)
    supervisor = supervisor_agent(intent, products, budget, copy)
    transaction = transaction_agent(intent, budget, products, copy)
    checkout = checkout_simulator(transaction, supervisor)

    # 2️⃣ Trust & Guardrails ✅ (MUST come before you build final JSON)
    guardrail_pass, issues = trust_guardrails(
        copy_output=copy,
        user_budget=intent.get("budget", 0),
        price_optimizer_output=budget,
        supervisor_output=supervisor
    )

    # 3️⃣ Packaging
    packaging_agent(intent, products, budget, copy, supervisor)

    # 4️⃣ Build Final JSON
    final_json = {
        "pipeline_run": datetime.now().isoformat(),
        "supervisor_agent": supervisor.get("agent_name", "Commerce Supervisor Agent"),
        "intent": intent.get("intent", user_request),
        "user_budget": intent.get("budget", 0),
        "guardrail_status": "pass" if guardrail_pass else "fail",
        "guardrail_issues": issues,
        "agents": {
            "intent_planner_agent": intent,
            "product_discovery_agent": products,
            "price_optimizer_agent": budget,
            "commerce_copywriter_agent": copy,
            "supervisor_agent": supervisor,
            "transaction_agent": transaction,
            "checkout_agent": checkout
        }
    }

    # 5️⃣ Save and display summary
    file_path = f"agentic_commerce_report_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.json"
    with open(file_path, "w", encoding="utf-8") as f:
        json.dump(final_json, f, indent=2)

    display(HTML(f"""
    <div style="border-left:5px solid #000;padding:16px;margin:10px 0;
                background-color:#f9f9f9;color:#000;font-family:'Segoe UI',sans-serif;">
      <h3 style="margin-top:0;">✅ Agentic Commerce Pipeline Summary</h3>
      <p><b>Intent:</b> {final_json['intent']}</p>
      <p><b>Budget:</b> ${final_json['user_budget']}</p>
      <p><b>Supervisor Agent:</b> {final_json['supervisor_agent']}</p>
      <p><b>Guardrails:</b> {final_json['guardrail_status']}</p>
      <p><b>JSON Report Path:</b> <code>{file_path}</code></p>
    </div>
    """))

    return final_json


## 7. Run Interactive Demo

The final cell prompts the user for a shopping request (e.g., “I want to set up a cozy home office under $500”).  
It then runs the full multi-agent commerce pipeline and displays the results.

🧠 Try entering different styles, categories, or budgets to see how the agents reason and whether the **trust guardrails** allow or block the transaction.


In [62]:

# =====================================================
# 7. RUN PIPELINE
# =====================================================
user_request = input("🧠 What would you like to shop for today? ")

# Example: "I want to set up a cozy home office under $500 with Scandinavian design."
final_summary = run_agentic_commerce_pipeline(user_request)


🧠 What would you like to shop for today? I want to set up a cozy home office under $500 with Scandinavian design
